In [ ]:
import os
import math
import openai
import glob
import subprocess
from pydub import AudioSegment


# 비디오에서 오디오 추출
def extract_audio_from_video(video_path, audio_path):
    command = [
        "ffmpeg",
        "-y",
        "-i",
        video_path,
        "-vn",
        audio_path,
    ]
    subprocess.run(command)


# 오디오 파일을 청크로 나누기
def cut_audio_in_chunks(audio_path, chunk_size, chunks_folder):
    # 오디오 파일 로드
    track = AudioSegment.from_mp3(audio_path)
    chunk_length = chunk_size * 60 * 1000  # 청크 크기를 밀리초로 변환
    chunks = math.ceil(len(track) / chunk_length)
    if not os.path.exists(chunks_folder):  # 청크 폴더가 존재하지 않으면 생성
        os.makedirs(chunks_folder)
    for i in range(chunks):  # 각 청크를 개별 파일로 내보내기
        start_time = i * chunk_length
        end_time = (i + 1) * chunk_length
        chunk = track[start_time:end_time]
        chunk.export(f"{chunks_folder}/jocoding-gpt_{i + 1:02d}.mp3", format="mp3")


# 폴더 내 모든 청크를 텍스트로 변환
def transcribe_chunks(chunk_folder, destination):
    files = glob.glob(f"{chunk_folder}/*.mp3")
    final_transcription = ""
    for file in files:
        with open(file, "rb") as audio_file:
            transcription = openai.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file,
            )
            final_transcription += transcription["text"]
    with open(destination, "w") as file:  # 기존 파일을 덮어씀
        file.write(final_transcription)


# 경로 설정
audio_path = "./files/jocoding-gpt.mp3"
chunks_folder = "./files/chunks"
chunk_size = 10  # 청크 크기 (분 단위)

# 오디오 파일을 청크로 나누기 (오디오 파일이 이미 존재한다고 가정)
cut_audio_in_chunks(audio_path, chunk_size, chunks_folder)

# 모든 오디오 청크를 텍스트로 변환
transcribe_chunks(chunks_folder, "transcription.txt")